In [1]:
#!conda install yapf -y

In [2]:
import os
import io
import boto3
import json
import base64
from yapf.yapflib.yapf_api import FormatCode

In [3]:
MODEL_ENDPOINT='nct-model-endpoint'
runtime = boto3.client('runtime.sagemaker')

In [4]:
#event='{"input":"int fromaws(){ return 8888; }"}'
event = '{"input":"Public int countFromArray(String target, ArrayList<String> array){ int count=0; for(String str: array){ if (target.equals(str)){ count++; }} return count; }"}'
data = event

print(data)

{"input":"Public int countFromArray(String target, ArrayList<String> array){ int count=0; for(String str: array){ if (target.equals(str)){ count++; }} return count; }"}


In [5]:
response = runtime.invoke_endpoint(EndpointName=MODEL_ENDPOINT,
                                   ContentType='application/json',
                                   Body=data)

result = response['Body'].read().decode()

print(result)

{"input": "Public int countFromArray(String target, ArrayList<String> array){ int count=0; for(String str: array){ if (target.equals(str)){ count++; }} return count; }", "output": ["def", "Public", "(", "target", ",", "array", ")", ":", "NEW_LINE", "INDENT", "count", "=", "0", "NEW_LINE", "for", "str", "in", "array", ":", "NEW_LINE", "INDENT", "if", "target", "==", "str", ":", "NEW_LINE", "INDENT", "count", "+=", "1", "NEW_LINE", "DEDENT", "DEDENT", "return", "count", "NEW_LINE", "DEDENT", "NEW_LINE"]}


In [67]:
jsonresult = json.loads(result)
out = jsonresult['output']
#out_list = [ token.replace('NEW_LINE','\n').replace('INDENT','\t').replace('DEDENT','') for token in out]
out_list = [ token for token in out]

out_formatted = ' '.join(out_list)
#out_formatted
#print(out_formatted)
out

['def',
 'Public',
 '(',
 'target',
 ',',
 'array',
 ')',
 ':',
 'NEW_LINE',
 'INDENT',
 'count',
 '=',
 '0',
 'NEW_LINE',
 'for',
 'str',
 'in',
 'array',
 ':',
 'NEW_LINE',
 'INDENT',
 'if',
 'target',
 '==',
 'str',
 ':',
 'NEW_LINE',
 'INDENT',
 'count',
 '+=',
 '1',
 'NEW_LINE',
 'DEDENT',
 'DEDENT',
 'return',
 'count',
 'NEW_LINE',
 'DEDENT',
 'NEW_LINE']

In [68]:
import re

new_out=[]
indent_list=[]
indent_count=0
indent_list_updated=[]

for token in out:
    if token == 'INDENT':
        indent_count+=1
        indent_list.append(indent_count)
    elif token == 'NEW_LINE':
        indent_list.append(99999)
    elif token == 'DEDENT':
        indent_count-=1
        indent_list.append(indent_count)
    else:
        indent_list.append(indent_count)
    

for b, indent in enumerate(indent_list):
    if indent_list[b-1] == indent:
        indent_list_updated.append(0)
    else:
        indent_list_updated.append(indent_list[b])
    
print(indent_list)
print(indent_list_updated)

assert len(indent_list) == len(indent_list_updated) == len(out)

for i in range(len(out)):
    if indent_list_updated[i] == 99999:
        new_out.append('\n')
    elif out[i] == 'INDENT' or out[i] == 'DEDENT':
        new_out.append(' '*4*indent_list_updated[i])
    else:
        new_out.append(' '*4*indent_list_updated[i]+out[i])

print(new_out)
#print(len(new_out))

assert len(out) == len(new_out)

new_out2=' '.join(new_out)
print(new_out2)

#out3=new_out2.split('\n')
for items in out3:
    print(items.strip())

[0, 0, 0, 0, 0, 0, 0, 0, 99999, 1, 1, 1, 1, 99999, 1, 1, 1, 1, 1, 99999, 2, 2, 2, 2, 2, 2, 99999, 3, 3, 3, 3, 99999, 2, 1, 1, 1, 99999, 0, 99999]
[0, 0, 0, 0, 0, 0, 0, 0, 99999, 1, 0, 0, 0, 99999, 1, 0, 0, 0, 0, 99999, 2, 0, 0, 0, 0, 0, 99999, 3, 0, 0, 0, 99999, 2, 1, 0, 0, 99999, 0, 99999]
['def', 'Public', '(', 'target', ',', 'array', ')', ':', '\n', '    ', 'count', '=', '0', '\n', '    for', 'str', 'in', 'array', ':', '\n', '        ', 'if', 'target', '==', 'str', ':', '\n', '            ', 'count', '+=', '1', '\n', '        ', '    ', 'return', 'count', '\n', '', '\n']
def Public ( target , array ) : 
      count = 0 
     for str in array : 
          if target == str : 
              count += 1 
               return count 
  

def Public ( target , array ) :
count = 0
for str in array :
if target == str :
count += 1
return count




In [75]:
def beautify_pycode(data):
    '''
        This code beautifies the model output in python.
    '''
    x=' '.join(data).split('NEW_LINE')
    
    indent_count = 0
    output=[]
    
    for items in x:
        o = []
        token = items.strip().split(' ')
        #print(token)
        for t in token:
            if t == 'INDENT':
                indent_count+=1
            elif t == 'DEDENT':
                indent_count-=1
            else:
                o.append(' '*3*indent_count+t)
        
        output.append(o)

    temp_out = [' '.join(o) for o in output]
    return str('\n'.join(temp_out))
 

print(beautify_pycode(out))

def Public ( target , array ) :
   count    =    0
   for    str    in    array    :
      if       target       ==       str       :
         count          +=          1
   return    count




In [129]:
#!conda install yapf -y

In [254]:
#from yapf.yapflib.yapf_api import FormatCode

In [255]:
#FormatCode(new_out2)

In [256]:
#!conda install autopep8 -y

In [261]:
import autopep8

In [267]:
ap_out_formatted = autopep8.fix_code(new_out2)
print(ap_out_formatted)

def __count_from_array(target, array):
      count = 0
     for str in array : 
          if target == str : 
              count += 1 
               return count 
  



In [263]:
autopep8.fix_lines(ap_out_formatted,options={})

AssertionError: 

In [136]:
#!conda install reindent -y

In [96]:
#import reindent